In [1]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [2]:
squad = squad.train_test_split(test_size=0.2)

In [6]:
squad["train"][0]

{'id': '56d07c41234ae51400d9c321',
 'title': 'Solar_energy',
 'context': 'Concentrating Solar Power (CSP) systems use lenses or mirrors and tracking systems to focus a large area of sunlight into a small beam. The concentrated heat is then used as a heat source for a conventional power plant. A wide range of concentrating technologies exists; the most developed are the parabolic trough, the concentrating linear fresnel reflector, the Stirling dish and the solar power tower. Various techniques are used to track the Sun and focus light. In all of these systems a working fluid is heated by the concentrated sunlight, and is then used for power generation or energy storage.',
 'question': 'What do Concentrating Solar Power technologies have in common?',
 'answers': {'text': ['a working fluid is heated by the concentrated sunlight'],
  'answer_start': [491]}}

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/home/salaam/Desktop/kenyAInspire---team-HassDonn/models/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [9]:
tokenized_squad = squad.map(
    preprocess_function, batched=True, remove_columns=squad["train"].column_names
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

2024-10-06 23:03:31.011663: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-06 23:03:31.094940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 23:03:31.124660: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 23:03:31.135175: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-06 23:03:31.192437: I tensorflow/core/platform/cpu_feature_guar

In [11]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [13]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(
    "distilbert/distilbert-base-uncased"
)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

  0%|          | 0/750 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 2.2348294258117676, 'eval_runtime': 7.5523, 'eval_samples_per_second': 132.41, 'eval_steps_per_second': 8.342, 'epoch': 1.0}
{'loss': 2.6847, 'grad_norm': 21.665542602539062, 'learning_rate': 6.666666666666667e-06, 'epoch': 2.0}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 1.6679327487945557, 'eval_runtime': 7.7839, 'eval_samples_per_second': 128.47, 'eval_steps_per_second': 8.094, 'epoch': 2.0}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 1.6113539934158325, 'eval_runtime': 7.7603, 'eval_samples_per_second': 128.861, 'eval_steps_per_second': 8.118, 'epoch': 3.0}
{'train_runtime': 285.3598, 'train_samples_per_second': 42.052, 'train_steps_per_second': 2.628, 'train_loss': 2.2511439208984374, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=2.2511439208984374, metrics={'train_runtime': 285.3598, 'train_samples_per_second': 42.052, 'train_steps_per_second': 2.628, 'total_flos': 1175877900288000.0, 'train_loss': 2.2511439208984374, 'epoch': 3.0})

In [40]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [41]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="./my_awesome_qa_model/checkpoint-750")
question_answerer(question=question, context=context)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.15568237006664276,
 'start': 58,
 'end': 95,
 'answer': '46 languages natural languages and 13'}